# 少数クラス

## データ読み込みと確認

In [ ]:
import pandas as pd

data = pd.read_csv("officialSource/qss-master/CAUSALITY/STAR.csv")

# データ外観
data.shape

# どこにどれくらい欠損がある？
data.isnull().sum()
# カウントする時は、classtypeをカウントすると良さそう

## 練習1
- データフレームに kinder という因子変数作成し、classtypeの整数値をわかりやすいラベルに変更
- race 変数を、アジア系、ネイティブ・アメリカンを others にまとめ、white, black, hispanic, others の4レベルの因子変数に変更。race変数に直接上書きする
- 欠損値除外をする

In [ ]:
data.loc[data["classtype"] == 1, "kinder"] = "small"
data.loc[data["classtype"] == 2, "kinder"] = "regular"
data.loc[data["classtype"] == 3, "kinder"] = "reg w/aid"

data.loc[data["race"].isin([3, 5, 6]), "race"] = "others"
data.loc[data["race"].isin([1]), "race"] = "white"
data.loc[data["race"].isin([2]), "race"] = "black"
data.loc[data["race"].isin([4]), "race"] = "hispanic"
data.groupby(by="race").count()

data_nonMissing = data[~data.isnull().any(axis=1)].copy()

## 練習2
- 4年次の「読解」と「算数」の成績について、少人数クラス所属の生徒と標準規模クラス所属の生徒とを比較する
    - 「読解」と「算数」の平均値を、「少人数クラス」「標準規模クラス」のグループ別で比較
    - 欠損レコードは除外しておく
- 結果の実質的な解釈を簡潔に述べる
- 推定された効果の大きさを理解するため、テスト点数の標準偏差を比べる

In [ ]:
temp = data.groupby(by="kinder").mean()[["g4math", "g4reading"]]
# 読解の平均の差
temp.loc["small", "g4reading"] - temp.loc["regular", "g4reading"]
# 算数の平均の差
temp.loc["small", "g4math"] - temp.loc["regular", "g4math"]

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white", color_codes=True)

# 標準偏差を出す
temp = data.groupby(by="kinder").std()[["g4math", "g4reading"]]
temp.loc["small", "g4reading"] - temp.loc["regular", "g4reading"]
temp.loc["small", "g4math"] - temp.loc["regular", "g4math"]
# ばらつきを確認
ax = sns.distplot(data_nonMissing.loc[data_nonMissing["kinder"]=="small", "g4reading"], label="small")
sns.distplot(data_nonMissing.loc[data_nonMissing["kinder"]=="regular", "g4reading"], ax=ax, label="regular")
ax.legend()
plt.show()

In [ ]:
# 模範解答では、個別に標準偏差は出さず、全体で出していたのでやってみる
data["g4reading"].std(skipna=True)
data["g4math"].std(skipna=True)

少数クラスと、標準規模クラスでは、読解の平均点の差が3.5点、算数の平均点の差が-0.34点であった。

標準偏差の値が、読解は52.4259、算数が43.0921である。このバラツキが起きる集団において、平均点の差が4点未満というのは、
充分に偶然起こりうる範囲内の点差と考えられるので、少数クラスによる介入効果は無いと考えられる。

## 練習3
- 少人数クラスと標準規模クラスの、点数の範囲を調べる
    - 66パーセンタイルで示されるハイスコア、33パーセンタイルで示されるロースコア同士を比較する
- この分析＋練習2の平均値による分析とを合わせて、なにか言えるか？

In [ ]:
small_df = data[data["kinder"]=="small"].copy()
regular_df = data[data["kinder"]=="regular"].copy()

small_df["g4reading"].quantile(0.33) - regular_df["g4reading"].quantile(0.33)
small_df["g4reading"].quantile(0.66) - regular_df["g4reading"].quantile(0.66)
small_df["g4math"].quantile(0.33) - regular_df["g4math"].quantile(0.33)
small_df["g4math"].quantile(0.66) - regular_df["g4math"].quantile(0.66)

3分位数の差もかなり小さく、ほぼ差がない状態。これは、分布にも差がない事を示しており、前述の平均値でも差がない事から、やはり幼稚園時代のクラスの規模が与えた変化は特になかった、と考えられる。

## 練習4
- 幼稚園時代に小規模クラスにいたかどうかのフラグと、その後も小規模クラスに何年所属し続けたかのデータがあるので、クロス集計表(人数)を作る
- 少人数クラスにより長く在籍した場合、テストの点数に大きな変化はあるだろうか？平均値、中央値を見比べてみよう

In [ ]:
# クロス集計表を作る
data.groupby(by=["kinder","yearssmall"]).count()["classtype"].unstack("yearssmall").to_csv("ch2/cross.csv", encoding="utf-8-sig")

# yearssmall別で、色々と指標を見てみる
data.groupby(by="yearssmall").describe()[["g4math", "g4reading"]]

# 分布図を書いてみる g4reading
ax = sns.distplot(data.loc[data["yearssmall"] == 0, "g4reading"].dropna(), label="0")
ax = sns.distplot(data.loc[data["yearssmall"] == 1, "g4reading"].dropna(), label="1", ax=ax)
ax = sns.distplot(data.loc[data["yearssmall"] == 2, "g4reading"].dropna(), label="2", ax=ax)
ax = sns.distplot(data.loc[data["yearssmall"] == 3, "g4reading"].dropna(), label="3", ax=ax)
ax = sns.distplot(data.loc[data["yearssmall"] == 4, "g4reading"].dropna(), label="4", ax=ax)
ax.legend()
plt.show()

# 分布図を書いてみる g4math
temp = data.loc[data["yearssmall"] == 0, "g4math"]
ax = sns.distplot(temp.dropna(), label="0")
ax = sns.distplot(data.loc[data["yearssmall"] == 1, "g4math"].dropna(), label="1", ax=ax)
ax = sns.distplot(data.loc[data["yearssmall"] == 2, "g4math"].dropna(), label="2", ax=ax)
ax = sns.distplot(data.loc[data["yearssmall"] == 3, "g4math"].dropna(), label="3", ax=ax)
ax = sns.distplot(data.loc[data["yearssmall"] == 4, "g4math"].dropna(), label="4", ax=ax)
ax.legend()
plt.show()

## 練習5 「STARプロジェクトで人種間の学力差はうまったのか？」を調べる
- classtype別の、白人orマイノリティの平均点比較をする

In [ ]:
del data["race_class"]

data.loc[data["race"]=="white", "race_class"] = "white"
data.loc[data["race"].isin(["black", "hispanic"]), "race_class"] = "minority"
temp = data.groupby(by=["kinder", "race_class"]).mean().unstack("race_class")
temp

temp.loc["regular", ("g4reading", "white")] - temp.loc["regular", ("g4reading", "minority")]
temp.loc["small", ("g4reading", "white")] - temp.loc["small", ("g4reading", "minority")]
temp.loc["regular", ("g4math", "white")] - temp.loc["regular", ("g4math", "minority")]
temp.loc["small", ("g4math", "white")] - temp.loc["small", ("g4math", "minority")]
# 分布状況を把握
data.groupby(by=["kinder", "race_class"]).describe()[["g4reading"]]
data.groupby(by=["kinder", "race_class"]).describe()[["g4math"]]


# 毎度、分布を見ておく
# minority の読解点数の分布
ax = sns.distplot(data.loc[(data["race_class"] == "minority") & (data["kinder"] == "small"), "g4reading"].dropna(), label="small")
ax = sns.distplot(data.loc[(data["race_class"] == "minority") & (data["kinder"] == "regular"), "g4reading"].dropna(), label="regular", ax=ax)
ax = sns.distplot(data.loc[(data["race_class"] == "minority") & (data["kinder"] == "reg w/aid"), "g4reading"].dropna(), label="reg w/aid", ax=ax)
ax.legend()
plt.show()



読解力においては、標準クラスの白人とマイノリティの点差が36点なのに対し、小規模クラスでは28点と、8点ほど縮まっている。同様の計算をすると、算数ではほぼ変化無し。
このことから、読解力においては、マイノリティの学習効果は上がる傾向がある。